In [123]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [24]:
CLIENT_ID = 'ID' # your Foursquare ID
CLIENT_SECRET = 'Secret' # your Foursquare Secret
VERSION = '20191222'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z
CLIENT_SECRET:T


In [137]:
univ=pd.read_csv('univer.csv')
univ

,name,code,lattitude,longitude,rent,students
0,Lomonosov Moscow State University,MSU,55.703092,37.530772,47100,40000
1,Higher School of Economics,HSE,55.761751,37.632456,60000,40000
2,MGIMO University,MGIMO,55.671999,37.488288,40300,6000
3,The Russian Presidential Academy of National E...,RANEPA,55.665107,37.479167,37700,6500
4,RUDN University,RUDN,55.652011,37.499527,37700,31500
5,Plekhanov Russian University of Economics,REA,55.727510,37.627605,37700,10800
6,Bauman Moscow State Technological University,BMSTU†,55.766029,37.684518,47100,19000
7,Moscow Pedagogical State University,MGPU,55.732533,37.576423,115600,26000
8,Kutafin Moscow State Law University,MSAL,55.762204,37.585919,49400,16000
9,Sechenov First Moscow State Medical University,1stMED,55.731687,37.571696,49500,15000


In [94]:
latitude=55.73
longitude=37.65
#Location of Moscow State University
moscow_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map (this code was copypasted, but dont blame me, i know how it works xD )
for lat, lng, name, code in zip(univ['lattitude'], univ['longitude'], univ['name'], univ['code']):
    label = '{}, {}'.format(name, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(moscow_map)

moscow_map

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['University', 
                  'University Latitude', 
                  'University Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
venues=getNearbyVenues(names=univ['code'],
                                   latitudes=univ['lattitude'],
                                   longitudes=univ['longitude'],
                                   radius=500
                                  )
venues.shape

(618, 7)

In [43]:
venues.head()

,University,University Latitude,University Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MSU,55.703092,37.530772,Парк МГУ,55.700361,37.534282,Park
1,MSU,55.703092,37.530772,MSU Fountain (Фонтан МГУ),55.705700,37.535262,Fountain
2,MSU,55.703092,37.530772,The Earth Science Museum at Moscow State Unive...,55.702775,37.531014,Science Museum
3,MSU,55.703092,37.530772,Площадь М. В. Ломоносова,55.701181,37.527626,Plaza
4,MSU,55.703092,37.530772,Университетская площадь,55.706092,37.536446,Plaza


In [51]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 182 uniques categories.


In [44]:
#let's look at the most common category, there are too many of them
rating=venues[['Venue Category','Venue']].groupby('Venue Category').count()
rating

,Venue
Venue Category,
Adult Boutique,2
American Restaurant,3
Arcade,4
Art Gallery,3
Art Museum,4
...,...
Wings Joint,1
Women's Store,1
Yoga Studio,8


In [54]:
#Not informative enough, let's sort it and pick top 25
rating.sort_values(by=['Venue'], ascending=False).head(25)

,Venue
Venue Category,
Coffee Shop,49
Café,23
Bakery,19
Gym / Fitness Center,15
Caucasian Restaurant,14
Italian Restaurant,13
Park,12
Cosmetics Shop,12
Restaurant,10


And here we are, the most popular one is coffee shop \
That is not enough, in fact, because we also have "CAFE" or "BAKERY, which is also okay \
So we have to save all food-related venues \
The most fascinating thing here is the fact, that caucasian resturants are at the 5th place \
Have never thought they are so common in Moscow

Okay, now we can get rid of those we don't need \
But we have to be careful, because we could also use some information about parks and supermarkets, as soon as they are good places to put a coffee station there.



In [84]:
venues_filtered=venues[
    (venues['Venue Category']=='Coffee Shop')
    | (venues['Venue Category']=='Café')
    | (venues['Venue Category']=='Bakery')
    | (venues['Venue Category']=='Caucasian Restaurant')
    | (venues['Venue Category']=='Italian Restaurant')
    | (venues['Venue Category']=='Park')
    | (venues['Venue Category']=='Restaurant')
    | (venues['Venue Category']=='Supermarket')
    | (venues['Venue Category']=='Fast Food Restaurant')
    | (venues['Venue Category']=='Beer Bar')
    | (venues['Venue Category']=='Wine Bar')
    | (venues['Venue Category']=='Sandwich Place')
    | (venues['Venue Category']=='Bar')
]

In [85]:
venues_filtered.reset_index(drop=True, inplace=True)

In [86]:
venues_filtered

,University,University Latitude,University Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MSU,55.703092,37.530772,Парк МГУ,55.700361,37.534282,Park
1,HSE,55.761751,37.632456,Хачапури,55.761583,37.635856,Caucasian Restaurant
2,HSE,55.761751,37.632456,Double B Coffee & Tea,55.761527,37.631717,Coffee Shop
3,HSE,55.761751,37.632456,Beer Gik Craft Beer Store,55.762208,37.635048,Beer Bar
4,HSE,55.761751,37.632456,Кулинарная лавка братьев Караваевых,55.761291,37.632562,Bakery
...,...,...,...,...,...,...,...
181,GUU,55.713249,37.816150,Perekrestok (Перекресток),55.710758,37.812022,Supermarket
182,GUU,55.713249,37.816150,KFC,55.712680,37.815737,Fast Food Restaurant
183,GUU,55.713249,37.816150,Coffee Like,55.712536,37.815719,Coffee Shop
184,GUU,55.713249,37.816150,Кофе'ок,55.713491,37.815854,Coffee Shop


In [87]:
len(venues_filtered)

186

In [97]:
coffee=[]
place=[]
food=[]
for i in range(len(venues_filtered)):
    if venues_filtered.loc[i]['Venue Category']=='Coffee Shop':
        coffee.append(1)
        place.append(0)
        food.append(0)
    elif venues_filtered.loc[i]['Venue Category']=='Park' or venues_filtered.loc[i]['Venue Category']=='Supermarket':
        coffee.append(0)
        place.append(1)
        food.append(0)
    else:
        coffee.append(0)
        place.append(0)
        food.append(1)
venues_filtered['coffee']=coffee.copy()
venues_filtered['place']=place.copy()
venues_filtered['food']=food.copy()

venues_filtered

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

,University,University Latitude,University Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,coffee,place,food
0,MSU,55.703092,37.530772,Парк МГУ,55.700361,37.534282,Park,0,1,0
1,HSE,55.761751,37.632456,Хачапури,55.761583,37.635856,Caucasian Restaurant,0,0,1
2,HSE,55.761751,37.632456,Double B Coffee & Tea,55.761527,37.631717,Coffee Shop,1,0,0
3,HSE,55.761751,37.632456,Beer Gik Craft Beer Store,55.762208,37.635048,Beer Bar,0,0,1
4,HSE,55.761751,37.632456,Кулинарная лавка братьев Караваевых,55.761291,37.632562,Bakery,0,0,1
...,...,...,...,...,...,...,...,...,...,...
181,GUU,55.713249,37.816150,Perekrestok (Перекресток),55.710758,37.812022,Supermarket,0,1,0
182,GUU,55.713249,37.816150,KFC,55.712680,37.815737,Fast Food Restaurant,0,0,1
183,GUU,55.713249,37.816150,Coffee Like,55.712536,37.815719,Coffee Shop,1,0,0
184,GUU,55.713249,37.816150,Кофе'ок,55.713491,37.815854,Coffee Shop,1,0,0


In [103]:
back=venues_filtered[['University','coffee','place','food']].groupby('University').sum()
#getting an overview for every university

In [104]:
back.reset_index(inplace=True)
#now we have everything we need, let's clusterize it
back

,University,coffee,place,food
0,1stMED,1,3,10
1,BMSTU†,2,1,8
2,GUBKIN,2,2,4
3,GUU,3,1,2
4,HSE,8,0,14
5,MAI,1,1,5
6,MGIMO,2,0,3
7,MGPU,1,3,5
8,MIIT,0,2,2
9,MIREA,4,3,9


In [115]:
x=back[back['University']=='MSU']['place']
int(x)

1

In [220]:
data=univ[['code','rent','students']]

data

,code,rent,students
0,MSU,47100,40000
1,HSE,60000,40000
2,MGIMO,40300,6000
3,RANEPA,37700,6500
4,RUDN,37700,31500
5,REA,37700,10800
6,BMSTU†,47100,19000
7,MGPU,115600,26000
8,MSAL,49400,16000
9,1stMED,49500,15000


In [221]:
coffee=[]
place=[]
food=[]
for i in range(len(data)):
    coffee.append(int(back[back['University']==data.loc[i]['code']]['coffee']))
    place.append(int(back[back['University']==data.loc[i]['code']]['place']))
    food.append(int(back[back['University']==data.loc[i]['code']]['food']))
data['coffee']=coffee.copy()
data['place']=place.copy()
data['food']=food.copy()

data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,code,rent,students,coffee,place,food
0,MSU,47100,40000,0,1,0
1,HSE,60000,40000,8,0,14
2,MGIMO,40300,6000,2,0,3
3,RANEPA,37700,6500,4,1,2
4,RUDN,37700,31500,0,0,5
5,REA,37700,10800,5,0,9
6,BMSTU†,47100,19000,2,1,8
7,MGPU,115600,26000,1,3,5
8,MSAL,49400,16000,2,2,10
9,1stMED,49500,15000,1,3,10


In [222]:
data['rent']/=data['rent'].max()
data['students']/=data['students'].max()
loc_max=max(data['coffee'].max(),data['place'].max(),data['food'].max())
data['coffee']/=loc_max
data['place']/=loc_max
data['food']/=loc_max

data

,code,rent,students,coffee,place,food
0,MSU,0.407439,1.0000,0.000000,0.066667,0.000000
1,HSE,0.519031,1.0000,0.533333,0.000000,0.933333
2,MGIMO,0.348616,0.1500,0.133333,0.000000,0.200000
3,RANEPA,0.326125,0.1625,0.266667,0.066667,0.133333
4,RUDN,0.326125,0.7875,0.000000,0.000000,0.333333
5,REA,0.326125,0.2700,0.333333,0.000000,0.600000
6,BMSTU†,0.407439,0.4750,0.133333,0.066667,0.533333
7,MGPU,1.000000,0.6500,0.066667,0.200000,0.333333
8,MSAL,0.427336,0.4000,0.133333,0.133333,0.666667
9,1stMED,0.428201,0.3750,0.066667,0.200000,0.666667


In [233]:
#CLUSTERING

# set number of clusters
kclusters = 3# because we have only 20 points and have to figure out who is good

data_clustering = data.drop('code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0,n_init=20).fit(data_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)
print(set(kmeans.labels_))

[2 0 1 1 2 1 1 2 1 1 1 1 0 0 1 1 1 1 1 1]
{0, 1, 2}


In [234]:
# add clustering labels
result=univ.copy()

result.insert(0, 'Cluster Labels', kmeans.labels_)

coffee=[]
place=[]
food=[]
for i in range(len(result)):
    coffee.append(int(back[back['University']==result.loc[i]['code']]['coffee']))
    place.append(int(back[back['University']==result.loc[i]['code']]['place']))
    food.append(int(back[back['University']==result.loc[i]['code']]['food']))
result['coffee']=coffee.copy()
result['place']=place.copy()
result['food']=food.copy()


result

,Cluster Labels,name,code,lattitude,longitude,rent,students,coffee,place,food
0,2,Lomonosov Moscow State University,MSU,55.703092,37.530772,47100,40000,0,1,0
1,0,Higher School of Economics,HSE,55.761751,37.632456,60000,40000,8,0,14
2,1,MGIMO University,MGIMO,55.671999,37.488288,40300,6000,2,0,3
3,1,The Russian Presidential Academy of National E...,RANEPA,55.665107,37.479167,37700,6500,4,1,2
4,2,RUDN University,RUDN,55.652011,37.499527,37700,31500,0,0,5
5,1,Plekhanov Russian University of Economics,REA,55.727510,37.627605,37700,10800,5,0,9
6,1,Bauman Moscow State Technological University,BMSTU†,55.766029,37.684518,47100,19000,2,1,8
7,2,Moscow Pedagogical State University,MGPU,55.732533,37.576423,115600,26000,1,3,5
8,1,Kutafin Moscow State Law University,MSAL,55.762204,37.585919,49400,16000,2,2,10
9,1,Sechenov First Moscow State Medical University,1stMED,55.731687,37.571696,49500,15000,1,3,10


In [235]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(result['lattitude'], result['longitude'], result['code'], result['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [236]:
#okay, what do we have here?
result[result['Cluster Labels']==0]

#These guys, as we can see, are bad for us - too many competitors

,Cluster Labels,name,code,lattitude,longitude,rent,students,coffee,place,food
1,0,Higher School of Economics,HSE,55.761751,37.632456,60000,40000,8,0,14
12,0,MIREA†Ч Russian Technological University,MIREA,55.661905,37.477617,37700,40000,4,3,9
13,0,Russian State University for the Humanities,RSUH,55.777178,37.595495,50500,25000,9,1,15


In [237]:
result[result['Cluster Labels']==1]
#These guys are middle-class, but I'm looking for a love suprime))


,Cluster Labels,name,code,lattitude,longitude,rent,students,coffee,place,food
2,1,MGIMO University,MGIMO,55.671999,37.488288,40300,6000,2,0,3
3,1,The Russian Presidential Academy of National E...,RANEPA,55.665107,37.479167,37700,6500,4,1,2
5,1,Plekhanov Russian University of Economics,REA,55.727510,37.627605,37700,10800,5,0,9
6,1,Bauman Moscow State Technological University,BMSTU†,55.766029,37.684518,47100,19000,2,1,8
8,1,Kutafin Moscow State Law University,MSAL,55.762204,37.585919,49400,16000,2,2,10
9,1,Sechenov First Moscow State Medical University,1stMED,55.731687,37.571696,49500,15000,1,3,10
10,1,Synergy University,Synergy,55.811087,37.510359,41100,18000,2,0,4
11,1,Moscow State University of Railway Engineering,MIIT,55.789306,37.602056,43500,17000,0,2,2
14,1,National University of Oil and Gas,GUBKIN,55.692282,37.555094,36200,11000,2,2,4
15,1,Pirogov Russian National Research Medical Univ...,Pirogov†,55.647256,37.490287,41100,9000,0,1,0


In [238]:
result[result['Cluster Labels']==2]
#Hooray - these guys are our future victims

,Cluster Labels,name,code,lattitude,longitude,rent,students,coffee,place,food
0,2,Lomonosov Moscow State University,MSU,55.703092,37.530772,47100,40000,0,1,0
4,2,RUDN University,RUDN,55.652011,37.499527,37700,31500,0,0,5
7,2,Moscow Pedagogical State University,MGPU,55.732533,37.576423,115600,26000,1,3,5
